In [1]:
from parsing import load_problem, load_solution
from graph_representation import Graph, get_random_starting_solution
import time
from copy import deepcopy

In [2]:
# weryfikuje poprawność sortowania
def verify_toposort(G):
    T = G.topological_order
    assert T is not None
    assert len(T) == len(G.vertices)
    for i in xrange(len(T)):
        v = T[i]
        for s in G.graph[v].successors:
            s = (v[0] + s[0],) + s[1:]
            if s in T[:i]:
                return False
    return True

In [3]:
def print_solution(G):
    for i in xrange(G.m):
        print G.solution[i]

In [4]:
def check_time(problem_name):
    n, m, problem = load_problem("instances/" + problem_name + ".fjs")
    init_solution = load_solution("solutions/" + problem_name + ".txt")
    graph = Graph(n, m, problem, init_solution)
    return graph.min_cycle_time()

In [4]:
import os
fnames = map(lambda x: x[:-4], sorted(os.listdir('instances')))

In [5]:
for name in fnames[:21]:
    print name, check_time(name)

Barnes_mt10c1 769.0
Barnes_mt10cc 728.0
Barnes_mt10x 712.0
Barnes_mt10xx 755.0
Barnes_mt10xxx 734.0
Barnes_mt10xy 754.0
Barnes_mt10xyz 671.0
Barnes_setb4c9 910.0
Barnes_setb4cc 902.0
Barnes_setb4x 914.0
Barnes_setb4xx 916.5
Barnes_setb4xxx 922.0
Barnes_setb4xy 907.0
Barnes_setb4xyz 902.0
Barnes_seti5c12 1164.0
Barnes_seti5cc 1133.0
Barnes_seti5x 1194.0
Barnes_seti5xx 1180.0
Barnes_seti5xxx 1182.0
Barnes_seti5xy 1132.0
Barnes_seti5xyz 1124.0


In [2]:
n, m, problem = load_problem("instances/Barnes_setb4xy.fjs")
init_solution = load_solution("solutions/Barnes_setb4xy.txt")
# init_solution = get_random_starting_solution(n, m, problem)
graph = Graph(n, m, problem, init_solution)
# graph.add_noise_to_solution(50)
initial_time = graph.min_cycle_time()
print "initial_time =", initial_time

initial_time = 907.0


In [4]:
G1 = deepcopy(graph)
G2 = deepcopy(graph)
G3 = deepcopy(graph)

In [5]:
execution_time_lb = G1.search_for_solution(5, cost_function=G1.lower_bound)

cycle_time_lb = G1.min_cycle_time()
print "execution_time_lb =", execution_time_lb
print "cycle_time_lb =", cycle_time_lb

searching...
0
((10, 15, 3), 9, 0)
((9, 15, 3), 10, 3)
1
((9, 15, 3), 9, 0)
((9, 15, 3), 9, 0)
found
execution_time_lb = 16.3485410213
cycle_time_lb = 1080.0


In [9]:
execution_time_lb = G2.search_for_solution(5, cost_function=G2.lower_bound_experimental)

cycle_time_lb = G2.min_cycle_time()
print "execution_time_lb =", execution_time_lb
print "cycle_time_lb =", cycle_time_lb

searching...
0
((2, 9, 7), 2, 11)
((2, 9, 7), 2, 10)
1
((0, 6, 2), 0, 1)
((0, 6, 2), 0, 1)
found
execution_time_lb = 22.6237578392
cycle_time_lb = 982.5


In [44]:
execution_time = G3.search_for_solution(5, cost_function=G3.min_cycle_time)

cycle_time = G3.min_cycle_time()
print "execution_time =", execution_time
print "cycle_time =", cycle_time

searching...
0
((1, 10, 5), 1, 6)
((1, 10, 5), 1, 10)
1
((2, 8, 5), 2, 10)
((2, 8, 5), 2, 8)
2
((0, 13, 8), 0, 9)
((0, 13, 8), 0, 14)
3
((4, 3, 4), 4, 3)
((4, 3, 4), 4, 1)
4
((5, 15, 2), 5, 4)
((5, 15, 2), 5, 3)
found
execution_time = 1480.93077898
cycle_time = 1700.0


In [12]:
for move in [((0, 9, 2), 0, 1)]:#graph.generate_neighborhood():
    a, b = graph.lower_bound_experimental(move), graph.min_cycle_time(move)
    print move, ' : ', a, b, a <= b

((0, 9, 2), 0, 1)  :  909 910.0 True


In [27]:
print_solution(graph)

[(12, 1), (9, 2), (1, 3), (10, 2), (2, 4), (3, 5), (6, 2), (14, 9), (15, 5), (5, 8), (4, 6), (7, 8), (13, 8), (11, 7), (8, 9)]
[(3, 1), (4, 1), (13, 2), (5, 5), (14, 8), (7, 4), (2, 5), (10, 5), (6, 6), (1, 9), (15, 9), (12, 8), (8, 8), (9, 10), (11, 9)]
[(14, 2), (5, 2), (7, 2), (15, 1), (12, 3), (3, 6), (13, 5), (9, 7), (1, 7), (8, 5), (10, 7), (11, 5), (2, 9), (4, 10), (6, 10)]
[(3, 2), (2, 3), (12, 2), (13, 3), (14, 7), (10, 3), (9, 6), (8, 4), (5, 7), (11, 2), (15, 7), (1, 10), (4, 8), (7, 10), (6, 9)]
[(14, 1), (9, 1), (3, 4), (4, 3), (1, 5), (6, 3), (7, 6), (2, 6), (11, 3), (8, 6), (5, 10), (15, 10), (10, 9), (12, 10), (13, 10)]
[(14, 3), (3, 3), (5, 4), (9, 4), (15, 2), (8, 3), (4, 5), (6, 4), (1, 8), (2, 7), (12, 6), (11, 4), (7, 9), (10, 8), (13, 9)]
[(2, 1), (4, 4), (5, 6), (9, 9), (6, 8), (8, 10)]
[(1, 1), (5, 1), (8, 2), (4, 2), (14, 6), (9, 5), (12, 4), (11, 1), (7, 7), (13, 7), (15, 8), (6, 7), (2, 10), (3, 10), (10, 10)]
[(8, 1), (2, 2), (5, 3), (14, 5), (6, 1), (7, 3),

In [4]:
for move in graph.generate_neighborhood():
    a, b = graph.lower_bound_experimental(move), graph.lower_bound(move)
    print move, ' : ', a, b, abs(a - b)

((0, 12, 1), 0, 0)  :  1189 1069 120
((0, 12, 1), 0, 1)  :  1189 1145 44
((0, 12, 1), 0, 2)  :  1217 1217 0
((0, 12, 1), 0, 3)  :  1222 1222 0
((0, 12, 1), 0, 4)  :  1245 1245 0
((0, 12, 1), 0, 5)  :  1328 1328 0
((0, 12, 1), 0, 6)  :  1386 1386 0
((0, 9, 2), 0, 0)  :  1189 1145 44
((0, 9, 2), 0, 1)  :  1189 1041 148
((0, 9, 2), 0, 2)  :  1189 1109 80
((0, 9, 2), 0, 3)  :  1189 1114 75
((0, 9, 2), 0, 4)  :  1189 1174 15
((0, 9, 2), 0, 5)  :  1263 1263 0
((0, 9, 2), 0, 6)  :  1321 1321 0
((0, 9, 2), 0, 7)  :  1416 1416 0
((0, 1, 3), 0, 0)  :  1189 1177 12
((0, 1, 3), 0, 1)  :  1189 1109 80
((0, 1, 3), 0, 2)  :  1189 942 247
((0, 1, 3), 0, 3)  :  1189 942 247
((0, 1, 3), 0, 4)  :  1189 942 247
((0, 1, 3), 0, 5)  :  1189 1020 169
((0, 1, 3), 0, 6)  :  1189 1078 111
((0, 1, 3), 0, 7)  :  1189 1173 16
((0, 1, 3), 0, 8)  :  1220 1220 0
((0, 1, 3), 0, 9)  :  1556 1556 0
((0, 1, 3), 0, 10)  :  1647 1647 0
((0, 10, 2), 0, 0)  :  1189 1074 115
((0, 10, 2), 0, 1)  :  1189 1025 164
((0, 10, 2), 0,

In [29]:
sorted(graph.generate_neighborhood(), key=graph.lower_bound)[:10]

[((11, 3, 7), 6, 3),
 ((10, 10, 1), 10, 1),
 ((7, 12, 4), 7, 7),
 ((11, 1, 4), 6, 1),
 ((10, 13, 1), 10, 1),
 ((10, 14, 10), 10, 2),
 ((1, 4, 1), 1, 1),
 ((1, 13, 2), 1, 2),
 ((9, 9, 3), 9, 1),
 ((9, 9, 3), 10, 2)]